# This notebook shows how to manage the events table with Spark

We first need to the kudu jars to the spark session

In [1]:
%%configure -f
{ "conf": {"spark.jars.packages": "org.apache.kudu:kudu-spark2_2.11:1.4.0" }}

In [2]:
sc

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
3,application_1517827649978_5175,spark,idle,Link,Link,✔


SparkSession available as 'spark'.
res1: org.apache.spark.SparkContext = org.apache.spark.SparkContext@58c7cef7

In [5]:
import org.apache.kudu.spark.kudu._
import org.apache.spark.sql.functions._

//we need to define a udf function for converting two strings into a map
val asMap = udf((keys:String, values: String) => keys.split("#").zip(values.split("#")).toMap)

//then we define a first dataframe df1 associated to the kudu table
val df1 = spark.sqlContext.read.options(Map("kudu.master" -> "master:7051","kudu.table" -> "Events3")).kudu

//the next dataframe contains an additional column of type Map containing a map for the two arrays: keys and values
//val df2 = df1.withColumn("tags", asMap($"attributesKeys",$"attributesValues"))

df1: org.apache.spark.sql.DataFrame = [source: string, ts: bigint ... 11 more fields]

In [69]:
val source = df1.select("source")

source: org.apache.spark.sql.DataFrame = [source: string]

In [70]:
val res = source.distinct.collect


res: Array[org.apache.spark.sql.Row] = Array([http://services.infoblu.it/BITW/data?id=teamd26lkc], [http://opendata.5t.torino.it/get_fdt], [http://159.122.135.24/daf/test/traffic_events?token=progettoDAF], [http://opendata.5t.torino.it/get_pk])

In [6]:
df1.printSchema()

root
 |-- source: string (nullable = false)
 |-- ts: long (nullable = false)
 |-- metric_id: string (nullable = false)
 |-- version: long (nullable = false)
 |-- id: string (nullable = true)
 |-- temporal_granularity: string (nullable = true)
 |-- event_certainty: double (nullable = false)
 |-- event_type_id: integer (nullable = false)
 |-- event_annotation: string (nullable = true)
 |-- location: string (nullable = true)
 |-- metric: double (nullable = false)
 |-- attributesKeys: string (nullable = true)
 |-- attributesValues: string (nullable = true)

In [73]:
source.groupBy("source").count().show()

+--------------------+--------+
|              source|   count|
+--------------------+--------+
|http://services.i...|67645216|
|http://opendata.5...|  145598|
|http://159.122.13...|     235|
|http://opendata.5...|   98328|
+--------------------+--------+

In [22]:
df1.filter("source == 'http://services.infoblu.it/BITW/data?id=teamd26lkc'").groupBy("ts").count().show()

+-------------+-----+
|           ts|count|
+-------------+-----+
|1519994000000| 9088|
|1520003960000| 9030|
|1520008880000| 8926|
|1520030180000| 8063|
|1520047520000| 4909|
|1520056100000| 5430|
|1520074040000| 8881|
|1520113280000| 8296|
|1520093840000| 9280|
|1520121800000| 8176|
|1520117720000| 7815|
|1520131400000| 6328|
|1520152640000| 6832|
|1520168000000| 9444|
|1520176220000| 9495|
|1520176700000| 9576|
|1520194340000| 8737|
|1520207240000| 7401|
|1520243180000| 8817|
|1519995680000| 9026|
+-------------+-----+
only showing top 20 rows

In [45]:
df1.filter("source == 'http://services.infoblu.it/BITW/data?id=teamd26lkc'").filter("ts == '1519994000000'")

res42: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [source: string, ts: bigint ... 11 more fields]

In [51]:
val df_sample = df1.filter("source == 'http://services.infoblu.it/BITW/data?id=teamd26lkc' and ts == '1519994000000'")
val df_sample_with_attributes = df_sample.withColumn("tags", asMap($"attributesKeys",$"attributesValues"))

df_sample_with_attributes: org.apache.spark.sql.DataFrame = [source: string, ts: bigint ... 12 more fields]

In [55]:
df_sample_with_attributes.take(10)

res50: Array[org.apache.spark.sql.Row] = Array([http://services.infoblu.it/BITW/data?id=teamd26lkc,1519994000000,it.gov.daf.kafkaProducers.eventConverters.InfoBluTrafficConverter.1519994000000.0,1,null,1.0,0,null,null,45.45764-9.12226,[B@75b8ecdc,srcCoordinates#dstCoordinates#sourceCode#tags#endCode#offset#metric#value,45.45764-9.12226#45.45869-9.12621#41149#sourceCode,endCode,offset#41150#1#speed#29,Map(srcCoordinates -> 45.45764-9.12226, dstCoordinates -> 45.45869-9.12621, sourceCode -> 41149, tags -> sourceCode,endCode,offset, endCode -> 41150, offset -> 1, metric -> speed, value -> 29)], [http://services.infoblu.it/BITW/data?id=teamd26lkc,1519994000000,it.gov.daf.kafkaProducers.eventConverters.InfoBluTrafficConverter.1519994000000.1,1,null,1.0,0,null,null,45.46137-9.13692,[B@77b9826...

In [56]:
val a = SystemE

<console>:29: error: value currentMillisec is not a member of object System
       val a = System.currentMillisec
                      ^


In [9]:
%%cleanup -f